# Notebook: View Dataset Statistics

This notebook is used to explore a given dataset.

## Packages

In [22]:
import pandas as pd
import tiktoken
import re

## Constants

In [23]:
DATASET_PATH = "reviews_dataset/balanced_reviews_sentences.csv"
LLM_MODEL_NAME = "gpt-3.5-turbo"

## Code

### General Statistics

In [24]:
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0.1,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language
0,0,3124,885663660,908561,0,Schon lange nicht mehr so schlecht gegessen,2023-04-10,michaellL5805JV,"Lissabon, Portugal","Suppe war nicht heiß, kümmerte den Service abe...",1.0,karlsruhe,Anders auf dem Turmberg,de,Schon lange nicht mehr so schlecht gegessen! A...,3,29201856-9893-4eec-a44a-82174cbd867c,de
1,1,338,909600304,742684,0,Keine Bedienung,2023-08-11,Patrick F,NaN,Keine BedienungSind 1 1/2 Stunden im Gastgarte...,2.0,münchen,Hofbräuhaus,de,Keine BedienungSind 1 1/2 Stunden im Gastgarte...,0,5ed24454-faec-4944-b968-b84b9fa5cb6e,de
2,2,988,884767828,965262,0,Ein Besuch in einem Düsseldorfer Brauhaus!,2023-04-04,Andreas K,"Leipzig, Deutschland",Im Vergleich zum Personal im Schlüssel und LOC...,3.0,düsseldorf,Uerige Hausbrauerei,de,Wir sind an einem Sonntagnachmittag durch mehr...,6,294fe7a8-c829-4706-89f7-cf88341c4828,de
3,3,1552,867801811,718273,0,gutes Essen in der Innenstadt,2022-11-09,Zwerg74,Aachen,Wir würden wiederkommen.,4.0,dortmund,Pfefferkorn,de,Das Essen war geschmacklich toll und Preis-Lei...,1,81343575-4711-4a44-bcb1-932dc3c2e0c0,de
4,4,3396,860042926,2049344,1,"Traditionscafé, das auch in Wie sein könnte, e...",2022-09-14,gmsorg,"Bad Homburg, Deutschland","LOC, eine beeindruckende Kuchen- und Tortenaus...",5.0,wiesbaden,Cafe Maldaner,de,Mitten in Wiesbaden in der belebten Fußgängerz...,1,b654d8c6-c609-48d6-9bbb-cac33caeff87,de
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,3995,3503,855090309,6541566,1,Leider nicht zu empfehlen,2022-08-19,Michael P,"Erkelenz, Deutschland","Ein weiteres absolutes NoGo war, dass sich wäh...",1.0,mönchengladbach,L'Osteria Mönchengladbach,de,Leider waren wir sehr unzufrieden und besuchen...,8,0764555d-88c3-4816-9177-c6ab11c87f9d,de
3996,3996,360,897549645,947503,0,Bei der lausigen Küche hilft die prächtige Dek...,2023-06-24,Happiness24765324521,"Paris, Frankreich","Sehr schade, da sitzt man im ""LOC"", einem der ...",2.0,münchen,Augustiner Stammhaus,de,"Sehr schade, da sitzt man im ""Arkadengarten"", ...",0,624cc1d8-2334-4e13-9a6b-c9767c68baf3,de
3997,3997,380,866848042,947503,2,Kurzer Imbiss,2022-11-01,509markuss,"Küsnacht, Schweiz",Gestresste überhebliche Kellner und das Essen ...,3.0,münchen,Augustiner Stammhaus,de,"Wir kannten das Restaurant von früher, aber al...",1,43e534c1-b1f6-4692-a9b6-71b398d431cf,de
3998,3998,3354,898452974,2640161,0,ein gelungener Abend,2023-06-27,garyaugs,"Augsburg, Deutschland",Im Rahmen einer Führung sind wir zu Besuch im ...,4.0,augsburg,Riegele Wirtshaus,de,Im Rahmen einer Führung sind wir zu Besuch im ...,0,fd93a399-1e35-466a-a8d7-54c6fd77b6bf,de


In [25]:
rating_counts = df['rating'].value_counts()
rating_counts

rating
1.0    800
2.0    800
3.0    800
4.0    800
5.0    800
Name: count, dtype: int64

In [26]:
rating_counts = df['city'].value_counts()
rating_counts

city
hamburg            433
leipzig            322
dresden            313
münchen            265
düsseldorf         195
münster            175
köln               174
frankfurt          173
bonn               172
stuttgart          169
nürnberg           154
berlin             144
augsburg           143
bremen             129
mönchengladbach    124
mannheim           121
wiesbaden          121
bochum             120
bielefeld          114
hannover           105
essen              103
karlsruhe           82
dortmund            66
wuppertal           55
duisburg            28
Name: count, dtype: int64

In [27]:
rating_counts = df['restaurant_name'].value_counts()
rating_counts

restaurant_name
Bullerei                    203
Mamma Mia                   175
Ratskeller                  174
Olive                        96
Schneider Bräuhaus           93
                           ... 
Thai Banyan Restaurant        4
Ola Lisboa                    3
Argentina - Steak & Mehr      3
bratar - Erbprinzen 27        2
Tokyo Sushi Bar               2
Name: count, Length: 122, dtype: int64

In [28]:
#df.groupby(['sentence_idx', 'review_id']).size().reset_index(name='count')

In [29]:
def count_words(text):
    words = re.findall(r'\w+', text)
    return len(words)

df['word_count'] = df['text'].apply(count_words)

In [30]:
# Standarddeviation
df['word_count'].std()


11.89603454557347

In [31]:
# Mean
df['word_count'].mean()

13.579

In [32]:
# Median
df['word_count'].median()

11.0

In [33]:
# Min, Max
df['word_count'].min(), df['word_count'].max()

(1, 204)

In [34]:
# Source: https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_string(string: str, model_name: str) -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [35]:
df['llm_token_count'] = df['text'].apply(num_tokens_from_string, model_name=LLM_MODEL_NAME)

In [36]:
df['llm_token_count'].min(), df['llm_token_count'].max()

(2, 344)

In [37]:
top_10_highest_tokens = df.nlargest(10, 'llm_token_count')
top_10_highest_tokens


,Unnamed: 0.1,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language,word_count,llm_token_count
3696,3696,2371,871086663,6931269,0,Bescheidener Beisel,2022-12-08,Torston,"Bochum, Deutschland","Seit Jahren eine gute Adresse , gerade für Gan...",2.0,bochum,Franz Ferdinand,de,"Seit Jahren eine gute Adresse , gerade für Gan...",0,49df65f6-5b51-4b3a-bf5f-11ddcae08be1,de,204,344
2699,2699,298,893195355,695667,0,"Trompetengalaktisch und sehr geil, am liebsten...",2023-06-03,rennboot,"Hamburg, Deutschland",An der Bar einen kleinen Aperitif und dann geh...,5.0,hamburg,Fischereihafen-Restaurant,de,"Moin Folks,können vier Gäste lügen, ich denke ...",1,f6268a7c-3f4f-4cfb-b918-9694ec8f7405,de,149,244
2055,2055,764,911246241,1341959,0,Sehr unfreundliches Personal,2023-08-18,Seaside48854597079,NaN,Zu diesem Zeitpunkt waren fast alle Tische im ...,1.0,frankfurt,Atschel,de,Diese Apfelweinwirtschaft kann ich leider nich...,1,b010cba9-3de3-4392-aa23-17ea251a9c56,de,112,214
1104,1104,3244,874893103,7322855,11,Erstklassig und unvergleichbar,2023-01-12,W1804MDmichaelm,NaN,Bereits vor unserem Besuch hörten wir öfter vo...,5.0,augsburg,SteakManufaktur,de,Bereits vor unserem Besuch hörten wir öfter vo...,0,47c984de-810d-405b-bc93-4787ff157a8d,de,108,195
2390,2390,1725,864019871,2049557,1,Schwer enttäuscht,2022-10-10,Y7454NFmikev,NaN,Schwer enttäuschtBratkartoffeln aus der fritös...,1.0,bremen,Schüttinger Gasthausbrauerei,de,Schwer enttäuschtBratkartoffeln aus der fritös...,0,28824f4e-bc50-4d64-92dc-9e9e43cf1433,de,109,195
3640,3640,1731,880313943,742761,0,Service sehr nett ….aber das Essen,2023-02-27,Erkenntnis,"Brinkum, Deutschland",Wir waren mit vier Personen dort und alle habe...,1.0,bremen,El Mundo,de,Wir waren mit vier Personen dort und alle habe...,0,36119430-28fa-4a7d-ad84-b21a875872a2,de,116,186
3022,3022,2705,883612634,3196403,7,Spitzenplatz bei Bewertung nicht gerechtfertig...,2023-03-26,Michael S,NaN,Die 3 bevorzugten griechischen Restaurants in ...,3.0,bonn,Olive,de,Stand März 2023 (Nr. 2 von 477 Restaurants in ...,2,4f4d654e-e729-4a11-b293-82bb3923d8dc,de,87,185
1622,1622,279,854552905,2271294,3,Leider nicht so zufrieden wie sonst zumeist …,2022-08-16,SanMic2804,"Aschaffenburg, Deutschland",ich liebe das Hard Rock Café eigentlich sehr- ...,3.0,hamburg,Hard Rock Cafe,de,ich liebe das Hard Rock Café eigentlich sehr- ...,0,b7175855-424c-44ef-b4ac-fa82d05a820d,de,118,173
1582,1582,1093,875273958,1340438,1,Preis-Leistung leider nicht gegeben,2023-01-16,thomaso893,"Böblingen, Deutschland","Ein wirklich schönes Lokal, der Karte nach etw...",3.0,düsseldorf,Fischhaus,de,"Ein wirklich schönes Lokal, der Karte nach etw...",0,381c21f5-b145-4398-8fd2-f47a8b99efba,de,97,168
1228,1228,443,904955470,718846,0,Nicht nur für Touristen,2023-07-23,SamuelRappaport,"München, Deutschland","So probierte ich etwa LOC, LOC oder Saures Lün...",4.0,münchen,Schneider Bräuhaus,de,Wir besuchen dieses Lokal in unredelmäßigen Ab...,1,0d19e177-5621-4a7b-9581-4c31d6d70d3e,de,81,162


### Count Tokens (for LLM)

In [38]:
df['llm_token_count'] = df['text'].apply(lambda x: num_tokens_from_string(x, LLM_MODEL_NAME))

In [39]:
pd.set_option('display.max_colwidth', None)

In [40]:
max_token_row = df[df['llm_token_count'] == df['llm_token_count'].max()]
print("Zeile mit den meisten Tokens:")
max_token_row#["llm_token_count"].values[0]

Zeile mit den meisten Tokens:


,Unnamed: 0.1,Unnamed: 0,review_id,restaurant_id,page_index,title,date,author_name,author_location,text,rating,city,restaurant_name,language_code,text_noanonymization,sentence_idx,id,detected_language,word_count,llm_token_count
3696,3696,2371,871086663,6931269,0,Bescheidener Beisel,2022-12-08,Torston,"Bochum, Deutschland","Seit Jahren eine gute Adresse , gerade für Ganz und Co im November Dezember Heute kam die Vorspeise zeitgleich mit dem Aperitif Der laff und uninspiriert daher kam - anschließend ließ der Service dann auf sich warten Die Gänsekeule war zu lang im Konvektomaten oder der Mikrowelle,Die Soße dick und uninspirierte mit teils uninspirierten, gegarten MaronenOb es geschmeckt habe wurde nicht gefragt , warum auch der Laden war ja voll Also fragte ich ob es denn nicht interessiere wie es denn so war, aber ja , natürlich …Die angebrachte Kritik wurde mit ich leite es so weiter beantwortet - erst auf Nachfrage oben es denn nicht abgebracht sei sich mal zu entschuldigen wurde uns beschieden na ja ok aber ich mach ja nur Service Auf dem angestrebten Niveau nicht haltbare, da ist jeder kleine Beisel in LOC besser und ehrlicherNach heute ist das Fazit nüchtern - 105 Euro für das für 2 Personen und diese Behandlung - Nicht nochmals — in LOC scheinen sich halbwegs gute Gastronomiebetriebe immer wieder gern selbstgerecht oder faul und gierig herunterzuwirtschaften manche schneller manche länger und nachhaltiger aber immer mit dem Blick auf den eigenen Geldbeutel Schade schade schade RESTAURANT_NAME - jetzt schick mit FF gebrandet - zu FF fällt mir nach dem heutigen Abend etwas anderes ein",2.0,bochum,Franz Ferdinand,de,"Seit Jahren eine gute Adresse , gerade für Ganz und Co im November Dezember Heute kam die Vorspeise zeitgleich mit dem Aperitif Der laff und uninspiriert daher kam - anschließend ließ der Service dann auf sich warten Die Gänsekeule war zu lang im Konvektomaten oder der Mikrowelle,Die Soße dick und uninspirierte mit teils uninspirierten, gegarten MaronenOb es geschmeckt habe wurde nicht gefragt , warum auch der Laden war ja voll Also fragte ich ob es denn nicht interessiere wie es denn so war, aber ja , natürlich …Die angebrachte Kritik wurde mit ich leite es so weiter beantwortet - erst auf Nachfrage oben es denn nicht abgebracht sei sich mal zu entschuldigen wurde uns beschieden na ja ok aber ich mach ja nur Service Auf dem angestrebten Niveau nicht haltbare, da ist jeder kleine Beisel in Wien besser und ehrlicherNach heute ist das Fazit nüchtern - 105 Euro für das für 2 Personen und diese Behandlung - Nicht nochmals — in Bochum scheinen sich halbwegs gute Gastronomiebetriebe immer wieder gern selbstgerecht oder faul und gierig herunterzuwirtschaften manche schneller manche länger und nachhaltiger aber immer mit dem Blick auf den eigenen Geldbeutel Schade schade schade Franz Ferdinand - jetzt schick mit FF gebrandet - zu FF fällt mir nach dem heutigen Abend etwas anderes ein",0,49df65f6-5b51-4b3a-bf5f-11ddcae08be1,de,204,344


In [41]:
with open("../prompt_template.txt", "r", encoding="utf-8") as file:
    prompt_template = file.read()

In [42]:
template_size = num_tokens_from_string(prompt_template, LLM_MODEL_NAME)
aspect_term_tag_size = num_tokens_from_string('<aspect-term aspect="ambient/interior" polarity="negative"></aspect-term>', LLM_MODEL_NAME)

In [43]:
template_size + 10 * max_token_row["llm_token_count"].values[0] + aspect_term_tag_size * 20

4092